<a href="https://colab.research.google.com/github/finiteautomata/hatespeech-classification/blob/main/notebooks/Hatespeech_Colab_TPU_Eval_Single.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de modelos de Hate Speech usando TPUs

En esta notebook haremos el entrenamiento de un modelo "pesado" de Hate Speech. La idea es usar las TPU para poder entrenar un BERT que toma ~ 10 hs en dos GPU

Ahora, a montar Drive

In [1]:
from google.colab import drive

drive.mount("/content/drive/")

Mounted at /content/drive/


Al toque perrito

In [2]:
repo_dir = "/content/drive/Shareddrives/HateSpeech/hatespeech-classification"
!cd $repo_dir && git pull
!cd $repo_dir && pip install -e . 

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 4), reused 8 (delta 2), pack-reused 0
Unpacking objects: 100% (14/14), done.
From https://github.com/finiteautomata/hatespeech-classification
   4137d6f..0420685  main       -> origin/main
Updating 4137d6f..0420685
Fast-forward
 notebooks/Hatespeech_Colab_TPU_Single.ipynb        |   7240 +
 notebooks/Results - 1 HS Single.ipynb              | 161151 ------------------
 ...ion.ipynb => Results - 1 plain detection.ipynb} |    219 +-
 notebooks/Results - 2 fine grained.ipynb           |    769 +
 notebooks/{ => misc}/Results - Context first.ipynb |     38 +-
 5 files changed, 8225 insertions(+), 161192 deletions(-)
 create mode 100644 notebooks/Hatespeech_Colab_TPU_Single.ipynb
 delete mode 100644 notebooks/Results - 1 HS Single.ipynb
 rename notebooks/{Results - 1 HS detection.ipynb => Results - 1 plain detection.ipynb} (95%)
 creat

## **NOTA IMPORTANTE**

Reiniciar notebook después de correr la anterior celda. Por algún motivo extraño, no reconoce el paquete que acabamos de instalar localmente

In [1]:
import json
import sys
import torch
from hatedetection import load_datasets
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from hatedetection.training import (
    tokenize, lengths
)
from hatedetection.metrics import compute_extended_category_metrics, extended_hate_categories
from hatedetection import BertForSequenceMultiClassification



def eval_hate_speech(
    model_name, output_path, context='none', eval_batch_size=16,
    ):
    """
    Evaluates a model
    """
    if context not in lengths.keys():
        print(f"context must be one of {lengths.keys()}")
        sys.exit(1)

    print(f"Model name: {model_name}")
    print("Context: ", context)
    _, _, test_dataset = load_datasets(add_body=True)

    model = BertForSequenceMultiClassification.from_pretrained(model_name, return_dict=True, num_labels=len(extended_hate_categories))

    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    print("Tokenizing and formatting \n\n")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.model_max_length = lengths[context]
    dataset = test_dataset

    dataset = dataset.map(
        lambda x: tokenize(tokenizer, x, context=context), batched=True, batch_size=eval_batch_size
    )

    def format_dataset(dataset):
        def get_category_labels(examples):
            return {
                'labels': torch.Tensor(
                    [examples[cat] for cat in extended_hate_categories]),
            }
        dataset = dataset.map(get_category_labels)
        dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
        return dataset

    dataset = format_dataset(dataset)

    print("Sanity check\n\n")

    print(tokenizer.decode(dataset[0]["input_ids"]), "\n\n")

    print("Predicting")

    training_args = TrainingArguments(
        output_dir=".",
        per_device_eval_batch_size=eval_batch_size,
    )


    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=lambda pred: compute_extended_category_metrics(dataset, pred),
    )

    preds = trainer.predict(dataset)

    serialized = {
        "predictions": preds.predictions.tolist(),
        "labels": preds.label_ids.tolist(),
        "metrics": preds.metrics
    }

    
    for k, v in serialized["metrics"].items():
        print(f"{k:<15} =  {v:.4f}")

    print(f"Saving at {output_path}")
    

    with open(output_path, "w+") as f:
        json.dump(serialized, f, indent=4)



Los salvamos a disco. Esto es porque tarda un huevo si lo hacemos 8 veces.

In [2]:
import os
import glob




models = glob.glob("/content/drive/Shareddrives/HateSpeech/models/bert-single-title-body_[0-9]*")

for model in models:
    names = model.split("_")

    if len(names) == 1:
        num = 1
    else:
        num = int(names[-1])

    output_path = f"/content/drive/Shareddrives/HateSpeech/results/single-title-body_{num}.json"

    print("*"*80)

    if os.path.exists(output_path):
        print(output_path, " already calculated -- skipping")
        continue
    eval_hate_speech(model, output_path, context="title+body")

********************************************************************************
/content/drive/Shareddrives/HateSpeech/results/single-title-body_1.json  already calculated -- skipping
********************************************************************************
/content/drive/Shareddrives/HateSpeech/results/single-title-body_2.json  already calculated -- skipping
********************************************************************************
/content/drive/Shareddrives/HateSpeech/results/single-title-body_3.json  already calculated -- skipping
********************************************************************************
/content/drive/Shareddrives/HateSpeech/results/single-title-body_4.json  already calculated -- skipping
********************************************************************************
/content/drive/Shareddrives/HateSpeech/results/single-title-body_5.json  already calculated -- skipping
***************************************************************************


Sanity check


[CLS] [USER] ojalá se funda y cague de hambre [SEP] mora godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " - puede marcar con justeza la fecha de inicio de la crisis : hace 10 meses que no brinda clases. en cambio, no logra avizorar el día que concluirá : " no sabemos cuándo empezamos, cuando volvemos a trabajar. ¿ ideas? ¡ ( tenemos ) miles! podemos hacer un show gratis al aire libre ". es por eso que, ante la falta de un horizonte, mora godoy decidió cerrar su escuela de danzas y rematar gran parte del vestuario para poder continuar adelante. " hace 20 años que estoy acá, pero llevo diez meses sin trabajar - le explicó la artista a guillermo andino en el programa informados de todo, por américa -. este es mi lugar de ensayo, de creación. y es muy duro. para poder seguir adelante me tengo que desprender del lugar y rematar gran parte del vestuario. que muchos bailarines y bailarines lo van a aprovechar porque el precio es muy bajo ".

eval_loss       =  0.0935
eval_calls_f1   =  0.6881
eval_women_f1   =  0.4182
eval_lgbti_f1   =  0.4650
eval_racism_f1  =  0.6892
eval_class_f1   =  0.5214
eval_politics_f1 =  0.6559
eval_disabled_f1 =  0.6023
eval_appearance_f1 =  0.7613
eval_criminal_f1 =  0.6469
eval_mean_f1    =  0.6054
eval_mean_precision =  0.6481
eval_mean_recall =  0.5727
eval_hate_precision =  0.7268
eval_hate_recall =  0.6617
eval_hate_f1    =  0.6927
eval_runtime    =  427.0465
eval_samples_per_second =  26.5620
init_mem_cpu_alloc_delta =  309106.0000
init_mem_gpu_alloc_delta =  0.0000
init_mem_cpu_peaked_delta =  18306.0000
init_mem_gpu_peaked_delta =  0.0000
test_mem_cpu_alloc_delta =  1574922.0000
test_mem_gpu_alloc_delta =  0.0000
test_mem_cpu_peaked_delta =  1235282.0000
test_mem_gpu_peaked_delta =  1108197376.0000
Saving at /content/drive/Shareddrives/HateSpeech/results/single-title-body_13.json
********************************************************************************
Model name: /content/drive/


Sanity check


[CLS] [USER] ojalá se funda y cague de hambre [SEP] mora godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " - puede marcar con justeza la fecha de inicio de la crisis : hace 10 meses que no brinda clases. en cambio, no logra avizorar el día que concluirá : " no sabemos cuándo empezamos, cuando volvemos a trabajar. ¿ ideas? ¡ ( tenemos ) miles! podemos hacer un show gratis al aire libre ". es por eso que, ante la falta de un horizonte, mora godoy decidió cerrar su escuela de danzas y rematar gran parte del vestuario para poder continuar adelante. " hace 20 años que estoy acá, pero llevo diez meses sin trabajar - le explicó la artista a guillermo andino en el programa informados de todo, por américa -. este es mi lugar de ensayo, de creación. y es muy duro. para poder seguir adelante me tengo que desprender del lugar y rematar gran parte del vestuario. que muchos bailarines y bailarines lo van a aprovechar porque el precio es muy bajo ".

eval_loss       =  0.0942
eval_calls_f1   =  0.6691
eval_women_f1   =  0.4174
eval_lgbti_f1   =  0.4790
eval_racism_f1  =  0.6903
eval_class_f1   =  0.4907
eval_politics_f1 =  0.6422
eval_disabled_f1 =  0.6008
eval_appearance_f1 =  0.7523
eval_criminal_f1 =  0.6606
eval_mean_f1    =  0.6003
eval_mean_precision =  0.6449
eval_mean_recall =  0.5659
eval_hate_precision =  0.7277
eval_hate_recall =  0.6589
eval_hate_f1    =  0.6916
eval_runtime    =  434.2769
eval_samples_per_second =  26.1190
init_mem_cpu_alloc_delta =  60231.0000
init_mem_gpu_alloc_delta =  0.0000
init_mem_cpu_peaked_delta =  18306.0000
init_mem_gpu_peaked_delta =  0.0000
test_mem_cpu_alloc_delta =  1505212.0000
test_mem_gpu_alloc_delta =  0.0000
test_mem_cpu_peaked_delta =  1244362.0000
test_mem_gpu_peaked_delta =  831414784.0000
Saving at /content/drive/Shareddrives/HateSpeech/results/single-title-body_14.json
********************************************************************************
Model name: /content/drive/Sh


Sanity check


[CLS] [USER] ojalá se funda y cague de hambre [SEP] mora godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " - puede marcar con justeza la fecha de inicio de la crisis : hace 10 meses que no brinda clases. en cambio, no logra avizorar el día que concluirá : " no sabemos cuándo empezamos, cuando volvemos a trabajar. ¿ ideas? ¡ ( tenemos ) miles! podemos hacer un show gratis al aire libre ". es por eso que, ante la falta de un horizonte, mora godoy decidió cerrar su escuela de danzas y rematar gran parte del vestuario para poder continuar adelante. " hace 20 años que estoy acá, pero llevo diez meses sin trabajar - le explicó la artista a guillermo andino en el programa informados de todo, por américa -. este es mi lugar de ensayo, de creación. y es muy duro. para poder seguir adelante me tengo que desprender del lugar y rematar gran parte del vestuario. que muchos bailarines y bailarines lo van a aprovechar porque el precio es muy bajo ".

eval_loss       =  0.0931
eval_calls_f1   =  0.6642
eval_women_f1   =  0.4051
eval_lgbti_f1   =  0.5030
eval_racism_f1  =  0.7064
eval_class_f1   =  0.4840
eval_politics_f1 =  0.6263
eval_disabled_f1 =  0.6067
eval_appearance_f1 =  0.7493
eval_criminal_f1 =  0.6506
eval_mean_f1    =  0.5995
eval_mean_precision =  0.6376
eval_mean_recall =  0.5690
eval_hate_precision =  0.7328
eval_hate_recall =  0.6622
eval_hate_f1    =  0.6957
eval_runtime    =  434.9634
eval_samples_per_second =  26.0780
init_mem_cpu_alloc_delta =  62445.0000
init_mem_gpu_alloc_delta =  0.0000
init_mem_cpu_peaked_delta =  18306.0000
init_mem_gpu_peaked_delta =  0.0000
test_mem_cpu_alloc_delta =  1514173.0000
test_mem_gpu_alloc_delta =  0.0000
test_mem_cpu_peaked_delta =  1241122.0000
test_mem_gpu_peaked_delta =  1133762560.0000
Saving at /content/drive/Shareddrives/HateSpeech/results/single-title-body_15.json
